# vina_demo

## Colab Setup

1. Check the runtime instance
2. Choose whether to use Google Drive (persistence) or temp `/content`
3. Install required Python packages
4. Get course files/data into the runtime
5. Mount Google Drive for persistent storage


### 1) Check the runtime


In [1]:
import sys, platform, os, textwrap
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("Working dir:", os.getcwd())

!arch

Python: 3.12.12
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Working dir: /content
x86_64


## 2) Choose whether to use Google Drive (persistence)

- If you want your edits to persist across sessions, use **Drive**.
- If you're just running a quick exercise, you can skip Drive and use the temporary Colab filesystem (`/content`).


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from pathlib import Path

# Change this folder name once; everything else uses it.
COURSE_DIR_NAME = "structbio_course"  # you can rename for your course

USE_DRIVE = True

if "drive" in str(Path("/content/drive")) and os.path.exists("/content/drive") and USE_DRIVE:
    ROOT = Path("/content/drive/MyDrive") / COURSE_DIR_NAME
else:
    ROOT = Path("/content") / COURSE_DIR_NAME

ROOT.mkdir(parents=True, exist_ok=True)

DATA_DIR = ROOT / "data"
(DATA_DIR).mkdir(exist_ok=True)

OUTPUTS_DIR = ROOT / "outputs"
(OUTPUTS_DIR).mkdir(exist_ok=True)

print("ROOT:", ROOT)
print("data:", DATA_DIR)
print("outputs:", OUTPUTS_DIR)


ROOT: /content/drive/MyDrive/structbio_course
data: /content/drive/MyDrive/structbio_course/data
outputs: /content/drive/MyDrive/structbio_course/outputs


### 2) Copy over the input data files and code needed for the vina demo

... and unzip the archive


In [4]:
import os, pathlib, urllib.request

RAW_URL = "https://raw.githubusercontent.com/vvoelz/chem5412-spring2026/main/data/assignment02.zip"
zip_path = pathlib.Path(DATA_DIR) / "assignment02.zip"

urllib.request.urlretrieve(RAW_URL, zip_path)
print("Saved to:", zip_path)

!unzip {zip_path}
!cp -r assignment02 {DATA_DIR}

print(f'Unzipped folder `assignment02` moved to {DATA_DIR}')

Saved to: /content/drive/MyDrive/structbio_course/data/assignment02.zip
Archive:  /content/drive/MyDrive/structbio_course/data/assignment02.zip
   creating: assignment02/
   creating: assignment02/input_files/
  inflating: assignment02/input_files/4ey7.pdb  
  inflating: assignment02/input_files/human_AChE_binders.csv  
  inflating: assignment02/input_files/4ey7_receptor_prepped.pdb  
   creating: assignment02/input_files/rivastigmine/
  inflating: assignment02/input_files/rivastigmine/rivastigmine_out.pdbqt  
  inflating: assignment02/input_files/rivastigmine/rivastigmine.pdbqt  
  inflating: assignment02/input_files/rivastigmine/rivastigmine.mol2  
  inflating: assignment02/input_files/rivastigmine/config_singledock  
  inflating: assignment02/input_files/rivastigmine/rivastigmine.smiles  
   creating: assignment02/input_files/huperzine/
  inflating: assignment02/input_files/huperzine/huperzine.smiles  
  inflating: assignment02/input_files/huperzine/huperzine.pdbqt  
  inflating: as

# Environment Setup

In these steps, a Python environment is prepared for you to run this example.

*Major Python packages used*

- RDKit https://rdkit.org/
- Molscrub https://github.com/forlilab/molscrub
- Meeko https://github.com/forlilab/Meeko
- ProDy http://www.bahargroup.org/prody/
- cctbx-base (for reduce2) https://github.com/cctbx/cctbx_project
- py3Dmol https://3dmol.org/

*Data*

- Phenix-project/geostd (for reduce2) https://github.com/phenix-project/geostd/

In [5]:
# @title (1) Install Condacolab (< 1min)
%%time

!pip install -q condacolab
import condacolab
condacolab.install()

#@markdown > An automatic restart of the kernel is expected after the execution of this block.
#@markdown >
#@markdown > Stay connected to the same runtime and proceed to the next code block!


⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...
CPU times: user 1.83 s, sys: 332 ms, total: 2.17 s
Wall time: 17.7 s


In [1]:
!conda info


     active environment : None
       user config file : /root/.condarc
 populated config files : /usr/local/.condarc
          conda version : 24.11.2
    conda-build version : not installed
         python version : 3.11.11.final.0
                 solver : libmamba (default)
       virtual packages : __archspec=1=broadwell
                          __conda=24.11.2=0
                          __glibc=2.35=0
                          __linux=6.6.105=0
                          __unix=0=0
       base environment : /usr/local  (writable)
      conda av data dir : /usr/local/etc/conda
  conda av metadata url : None
           channel URLs : https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
          package cache : /usr/local/pkgs
                          /root/.conda/pkgs
       envs directories : /usr/local/envs
                          /root/.conda/envs
               platform : linux-64
             user-agent :

In [44]:
# @title Step 1: Install Python packages
!conda install -c conda-forge numpy scipy rdkit gemmi meeko==0.6.1 vina autogrid cctbx-base

Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 26.1.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [24]:
# @title Step 2: Cloning data repository for Phenix Project geostd ...
# Phenix Project geostd (restraint) library
!git clone https://github.com/phenix-project/geostd.git geostd

Step 2: Cloning data repositories...
Cloning into 'geostd'...
remote: Enumerating objects: 300370, done.
remote: Counting objects: 100% (5709/5709), done.
remote: Compressing objects: 100% (5150/5150), done.
remote: Total 300370 (delta 627), reused 3055 (delta 559), pack-reused 294661 (from 2)
Receiving objects: 100% (300370/300370), 431.51 MiB | 11.90 MiB/s, done.
Resolving deltas: 100% (144112/144112), done.
Updating files: 100% (198142/198142), done.


In [36]:
# @title Step 3: Cloning data repository for prody from repo for numpy 2 compatibility
# temporarily check out
!git clone https://github.com/prody/ProDy.git prody

!cd prody; pip install .; cd ..

Cloning into 'prody'...
remote: Enumerating objects: 53654, done.
remote: Counting objects: 100% (2106/2106), done.
remote: Compressing objects: 100% (526/526), done.
remote: Total 53654 (delta 1863), reused 1586 (delta 1580), pack-reused 51548 (from 5)
Receiving objects: 100% (53654/53654), 163.36 MiB | 15.11 MiB/s, done.
Resolving deltas: 100% (36085/36085), done.
Updating files: 100% (751/751), done.
Processing /content/prody
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ProDy: filename=prody-2.6.1-cp311-cp311-linux_x86_64.whl size=37472128 sha256=a0837e8688303728f1ca34f09e785b4ed6246ecd6b5f43073aedbf6934304d55
  Stored in directory: /tmp/pip-ephem-wheel-cache-95h1v6c6/wheels/48/74/d2/6073201dec0af131a39053b75b1552f574ef2ba27a00d068cc
Successfully built ProDy
  Attempting uninstall: ProDy
    Found existing installation: ProDy 2.6.1
    Uninstalling ProDy-2.6.1:
      Success

In [34]:
# @title Step 4: Downloading autodock_gpu executable...
!wget -q --show-progress https://github.com/ccsb-scripps/AutoDock-GPU/releases/download/v1.6/adgpu-v1.6_linux_x64_cuda12_128wi

# !mv adgpu-v*_linux_x64_cuda12_128wi adgpu
!echo "Colab environment setup completed successfully!"

adgpu-v1.6_linux_x6 100%[===================>]  11.93M  60.1MB/s    in 0.2s    
Colab environment setup completed successfully!


In [49]:
!conda install rdkit rdkit-pypi

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ | / - \ | failed

PackagesNotFoundError: The following packages are not available from current channels:

  - rdkit-pypi

Current channels:

  - https://conda.anaconda.org/conda-forge

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [50]:
import

SyntaxError: invalid syntax (ipython-input-2125240012.py, line 1)

In [47]:
!pip install rdkit

ModuleNotFoundError: No module named 'rdkit'

In [40]:
# @title (3) Import Modules & Locate Command Line Scripts (< 1s)
%%time

!pip install rdkit

# Import modules
import sys, platform
from prody import *
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit, py3Dmol
print("rdkit version:", rdkit.__version__)
print("py3Dmol version:", py3Dmol.__version__)
from ipywidgets import interact, IntSlider
import ipywidgets, copy
from IPython.display import display, Markdown


# Helper functions
def locate_file(from_path = None, query_path = None, query_name = "query file"):

    if not from_path or not query_path:
        raise ValueError("Must specify from_path and query_path")

    possible_path = list(from_path.glob(query_path))

    if not possible_path:
        raise FileNotFoundError(f"Cannot find {query_name} from {from_path} by {query_path}")

    return_which = (
        f"using {query_name} at:\n"
        f"{possible_path[0]}\n"
    )
    print(return_which)

    return possible_path[0]


# Commandline scripts
scrub = locate_file(from_path = Path("/usr/local/bin"), query_path = "scrub.py", query_name = "scrub.py")
mk_prepare_ligand = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_prepare_ligand.py", query_name = "mk_prepare_ligand.py")
mk_prepare_receptor = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_prepare_receptor.py", query_name = "mk_prepare_receptor.py")
mk_export = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_export.py", query_name = "mk_export.py")


# Locate reduce2 in conda install prefix
full_py_version = platform.python_version()
major_and_minor = ".".join(full_py_version.split(".")[:2])
env_path = Path("/usr/local") # default conda install prefix on Colab
reduce2_path = f"lib/python{major_and_minor}/site-packages/mmtbx/command_line/reduce2.py"
reduce2 = locate_file(from_path = env_path, query_path = reduce2_path, query_name = "reduce2.py")


# Locate geostd in current path
geostd_path = locate_file(from_path = Path.cwd(), query_path = "geostd", query_name = "geostd")

#@markdown > Version of imported modules and the location of command line scripts will be reported to output.
#@markdown >
#@markdown > Make sure there are no errors and proceed to the next code block!


ModuleNotFoundError: No module named 'rdkit'

In [6]:
!apt-get update -qq
!apt-get install -y openbabel

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libboost-iostreams1.74.0 libinchi1 libmaeparser1 libopenbabel7
The following NEW packages will be installed:
  libboost-iostreams1.74.0 libinchi1 libmaeparser1 libopenbabel7 openbabel
0 upgraded, 5 newly installed, 0 to remove and 63 not upgraded.
Need to get 4,148 kB of archives.
After this operation, 19.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libboost-iostreams1.74.0 amd64 1.74.0-14ubuntu3 [245 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libinchi1 amd64 1.03+dfsg-4 [455 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmaeparser1 amd64 1.2.4-1build1 [88.

Install the **vina** Python package

In [27]:
!pip install vina

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.1 MB/s eta 0:00:00


**Install pandas**

In [30]:
!pip install pandas

Check to see if we have the **wget** CLI download tool

In [31]:
!which wget

/usr/bin/wget


# Verify the vina Python wrapper install

In [51]:
from vina import Vina

v = Vina(sf_name='vina')

# To prepare the receptor, we will use the **meeko** package, and **rdkit**

In [54]:
# 1) install meeko (and prody optional but helpful)
!pip -q install meeko prody
!pip -q install rdkit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 22.7 MB/s eta 0:00:00


In [57]:
!pip -q install gemmi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.1 MB/s eta 0:00:00


In [64]:
# 2) prepare receptor PDBQT
pdb_of_prepared_receptor = str(DATA_DIR / 'assignment02/input_files/4ey7_receptor_prepped.pdb')
print('pdb_of_prepared_receptor', pdb_of_prepared_receptor)
!ls {pdb_of_prepared_receptor}


pdb_of_prepared_receptor /content/drive/MyDrive/structbio_course/data/assignment02/input_files/4ey7_receptor_prepped.pdb
/content/drive/MyDrive/structbio_course/data/assignment02/input_files/4ey7_receptor_prepped.pdb


In [71]:
!mk_prepare_receptor.py --read_with_prody {pdb_of_prepared_receptor} -o receptor  --default_altloc A

@> 8298 atoms and 1 coordinate set(s) were parsed in 0.06s.
matched with excess inter-residue bond(s): A:69
matched with excess inter-residue bond(s): A:96
matched with excess inter-residue bond(s): A:161
matched with excess inter-residue bond(s): A:257
matched with excess inter-residue bond(s): A:261
matched with excess inter-residue bond(s): A:262
matched with excess inter-residue bond(s): A:266
matched with excess inter-residue bond(s): A:272
matched with excess inter-residue bond(s): A:409
matched with excess inter-residue bond(s): A:529
Traceback (most recent call last):
  File "/usr/local/bin/mk_prepare_receptor.py", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/meeko/cli/mk_prepare_receptor.py", line 610, in main
    polymer = Polymer.from_prody(
              ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/meeko/polymer.py", line 1331, in from_prody
    polymer = cls(
              ^^^^
  File "/

In [68]:
!mk_prepare_receptor.py --help

usage: mk_prepare_receptor.py [-h] [--read_pdb PDB_FILENAME]
                              [--read_pqr PQR_FILENAME] [-i MACROMOL_FILENAME]
                              [-o OUTPUT_BASENAME] [-p [PDBQT_FILENAME ...]]
                              [-j [JSON_FILENAME ...]]
                              [--write_pdb [PDB_FILENAME ...]]
                              [-g [GPF_FILENAME ...]]
                              [-v [VINA_BOX_FILENAME ...]]
                              [--debug_fn DEBUG_FN] [-n SET_TEMPLATE]
                              [-d DELETE_RESIDUES] [-b BLUNT_ENDS]
                              [--add_templates ADD_TEMPLATES]
                              [--cache_templates [CACHE_TEMPLATES]]
                              [--mk_config JSON_FILENAME] [-a]
                              [--default_altloc DEFAULT_ALTLOC]
                              [--wanted_altloc WANTED_ALTLOC] [-f FLEXRES]
                              [-t ROT_TERMINAL_GROUP]
                             

In [ ]:
from vina import Vina

v = Vina(sf_name='vina')

v.set_receptor('receptor.pdbqt')
v.set_ligand_from_file('ligand.pdbqt')

v.compute_vina_maps(center=[0,0,0], box_size=[20,20,20])
v.dock(exhaustiveness=8)
v.write_poses('out.pdbqt', n_poses=5)
